In [32]:
#!pip install catboost
#!pip install xgboost

In [33]:
#!pip install lightgbm

In [45]:

import numpy as np
import pandas as pd

df = pd.read_csv('C:\\Users\\amanr\\OneDrive\\Desktop\\LE.csv')
df = df [['Country','Year','Status','Life expectancy']]

In [46]:
df.head()

,Country,Year,Status,Life expectancy
0,Afghanistan,2015,Developing,65.0
1,Afghanistan,2014,Developing,59.9
2,Afghanistan,2013,Developing,59.9
3,Afghanistan,2012,Developing,59.5
4,Afghanistan,2011,Developing,59.2


In [47]:
from catboost import CatBoostRegressor, Pool
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

pool_train = Pool(X_train, y_train,
                  cat_features = ['Country','Year','Status'])

pool_test = Pool(X_test, cat_features = ['Country','Year','Status'])


#CatBoost

import time

start = time.time()

cbr = CatBoostRegressor(iterations=100,max_depth=2)

cbr.fit(pool_train)
y_pred = cbr.predict(X_test)

from sklearn.metrics import r2_score as RSquared


cb_rsquared = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for CatBoost: ", np.mean(cb_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)


Learning rate set to 0.304491
0:	learn: 7.9652996	total: 6.37ms	remaining: 631ms
1:	learn: 6.8750145	total: 15ms	remaining: 734ms
2:	learn: 6.3649201	total: 27.1ms	remaining: 875ms
3:	learn: 6.0458584	total: 36.8ms	remaining: 884ms
4:	learn: 5.7968748	total: 45.1ms	remaining: 856ms
5:	learn: 5.6849516	total: 55.2ms	remaining: 865ms
6:	learn: 5.5930909	total: 63.6ms	remaining: 846ms
7:	learn: 5.5501911	total: 71.5ms	remaining: 822ms
8:	learn: 5.5157724	total: 80.3ms	remaining: 812ms
9:	learn: 5.4922729	total: 89.3ms	remaining: 804ms
10:	learn: 5.4620773	total: 97.6ms	remaining: 790ms
11:	learn: 5.4609070	total: 106ms	remaining: 778ms
12:	learn: 5.4574215	total: 114ms	remaining: 765ms
13:	learn: 5.4314251	total: 122ms	remaining: 752ms
14:	learn: 5.4191324	total: 132ms	remaining: 747ms
15:	learn: 5.4029769	total: 146ms	remaining: 766ms
16:	learn: 5.3918803	total: 153ms	remaining: 745ms
17:	learn: 5.3861049	total: 161ms	remaining: 732ms
18:	learn: 5.3766558	total: 172ms	remaining: 735ms
19

In [48]:
#XGBoost
import xgboost as xgb
from sklearn import preprocessing
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

lbl = preprocessing.LabelEncoder()
#Country','Year','Status
X['Country'] = lbl.fit_transform(X['Country'].astype(str))
X['Year'] = lbl.fit_transform(X['Year'].astype(str))
X['Status'] = lbl.fit_transform(X['Status'].astype(str))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

start = time.time()
#X_train["Species"].astype("category")
xgbr = xgb.XGBRegressor()

xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)

xgb_rmse = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for XGBoost: ", np.mean(xgb_rmse))

end = time.time()
diff = end - start
print('Execution time:', diff)

R Squared for XGBoost:  0.9787312801955712
Execution time: 0.1608593463897705


In [49]:
import lightgbm
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']
obj_feat = list(X.loc[:, X.dtypes == 'object'].columns.values)

for feature in obj_feat:
    X[feature] = pd.Series(X[feature], dtype="category")

start = time.time()

lgbmr = lightgbm.LGBMRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)

lgbm_rsquared = np.sqrt(RSquared(y_test, y_pred))
print()
print("R squared for LightGBM: ", np.mean(lgbm_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)


R squared for LightGBM:  0.9767455250796998
Execution time: 0.09908795356750488


# 1. Community support/Performance/Want to tune - XGBOOST
# 2. Hurry to train/Performance/Dont want to tune - LightGBM
# 3. More categorical, GPU, Large data - CatBoost